Dans ce fichier, le but est d'analyser l'espace latent pour le sillon SC_SPeC_right des sujets de epilepsy_PBS

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.stats import ks_2samp
from scipy import stats
from sklearn.svm import OneClassSVM
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

/neurospin/dico/fred/Runs/01_betaVAE/Program/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/neurospin/dico/fred/Runs/01_betaVAE/Program/venv/lib/python3.12/site-packages/umap/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
Embeddings_ukb = pd.read_csv("/neurospin/tmp/fred/models/2025-11-10/SC-SPeC_right_UKB_16-17-31_42/Embeddings.csv")
Embeddings_epilepsy = pd.read_csv("/neurospin/tmp/fred/models/2025-11-10/SC-SPeC_right_UKB_16-17-31_42/epilepsy_PBS/Embeddings.csv")
Embeddings_hcp = pd.read_csv("/neurospin/tmp/fred/models/2025-11-10/SC-SPeC_right_UKB_16-17-31_42/hcp/Embeddings.csv")

In [3]:
Embeddings_ukb= np.array(Embeddings_ukb)
Embeddings_epilepsy= np.array(Embeddings_epilepsy)
Embeddings_hcp= np.array(Embeddings_hcp)

In [4]:
index_D = [19,28,31,34,41,43,44, 20,23,24,25,27,29,32,33,36,51,55,56]
index_G = [21,26,35,38,40,45, 22,30,37,39,42,46,47,48,49,50,52,53,54]

In [5]:
Embeddings_controls =Embeddings_epilepsy[0:19,:]
Embeddings_D=Embeddings_epilepsy[index_D,:]
Embeddings_G=Embeddings_epilepsy[index_G,:]

In [6]:
X_ukb= Embeddings_ukb[:,1:]
X_hcp= Embeddings_hcp[:,1:]
X_controls= Embeddings_controls[:,1:]
X_D= Embeddings_D[:,1:]
X_G= Embeddings_G[:,1:]

In [7]:
scaler= StandardScaler()
X1 = scaler.fit_transform(X_ukb)
X2= scaler.transform(X_hcp)
X3= scaler.transform(X_controls)
X4= scaler.transform(X_D)
X5= scaler.transform(X_G)

In [8]:
L= np.concatenate((X1,X2,X3,X4,X5), axis=0)

covariates = pd.DataFrame({
    'Site': ['UKB']*len(X1) 
            + ['hcp']*len(X2) 
            + ['epilepsy_PBS']*(len(X3)+len(X4)+len(X5)) 
})
covariates = pd.get_dummies(covariates)

covariates_train= covariates.head(len(X1)+len(X2)+len(X3))

covariates_test= covariates.iloc[len(X1)+len(X2)+len(X3):len(X1)+len(X2)+len(X3)+len(X4)+len(X5)]

L_train=np.concatenate((X1,X2,X3), axis=0)
L_test= np.concatenate((X4,X5), axis=0)

covariates = sm.add_constant(covariates)

L1_resid = np.zeros_like(L_train)
L2_resid = np.zeros_like(L_test)

for j in range(L_train.shape[1]):
    model = sm.OLS(L_train[:, j].astype(float), covariates_train.astype(float)).fit()
    L1_resid[:, j] = model.resid 
    L2_resid[:, j] = L_test[:,j] - model.predict(covariates_test)

X1 = L1_resid[:len(X1),:]
X2 = L1_resid[len(X1):len(X1)+len(X2),:]
X3= L1_resid[len(X1)+len(X2):len(X1)+len(X2)+len(X3),:]
X4= L2_resid[:len(X4),:]
X5=L2_resid[len(X4):,:]

In [9]:
reducer = umap.UMAP(n_components=2, random_state=42)
X1_umap = reducer.fit_transform(X1)
X2_umap = reducer.transform(X2)
X3_umap= reducer.transform(X3)
X4_umap=reducer.transform(X4)
X5_umap=reducer.transform(X5)

In [10]:
nu=0.02

In [11]:
clf = OneClassSVM(kernel='rbf', gamma='scale', nu=nu)
clf.fit(X1_umap)
y3 = clf.predict(X3_umap)
y4 = clf.predict(X4_umap)
y5 = clf.predict(X5_umap)

In [12]:
dic_control = {'subject': Embeddings_controls[:,0], 
                'prediction': y3
}
df_control = pd.DataFrame(dic_control)
df_control

,subject,prediction
0,control01,1
1,control02,1
2,control03,-1
3,control04,1
4,control05,-1
5,control06,1
6,control07,1
7,control08,-1
8,control09,1
9,control10,1


In [13]:
print("Proportion of normal:",np.mean(y3==1))

Proportion of normal: 0.6842105263157895


In [ ]:
dic_D = {'subject': Embeddings_D[:,0], 
            'prediction': y4
}
df_D = pd.DataFrame(dic_D)
df_D

,subject,prediction
0,sujet01_neg,-1
1,sujet08_neg,1
2,sujet10_neg,-1
3,sujet12_neg,1
4,sujet20_neg,-1
5,sujet22_neg,1
6,sujet23_neg,1
7,sujet02_pos,1
8,sujet04_pos,1
9,sujet05_pos,1


In [15]:
print("Proportion of normal:",np.mean(y4==1))


Proportion of normal: 0.6842105263157895


In [16]:
dic_G = {'subject': Embeddings_G[:,0], 
                'prediction': y5
}
df_G = pd.DataFrame(dic_G)
df_G

,subject,prediction
0,sujet03_neg,1
1,sujet07_neg,1
2,sujet13_neg,-1
3,sujet15_neg,1
4,sujet16_neg,1
5,sujet26_neg,-1
6,sujet03_pos,1
7,sujet09_pos,1
8,sujet14_pos,1
9,sujet15_pos,-1


In [17]:
print("Proportion of normal:",np.mean(y5==1))


Proportion of normal: 0.631578947368421
